Amazon SA Apple

In [36]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

urls = []
driver = webdriver.Chrome()
mainurl = 'https://www.amazon.sa/stores/page/CD3AFAED-67E7-44C6-A341-3F36FE846F59?ingress=2&visitId=92d9e18a-005d-4290-9268-02cabad7c12e&ref_=ast_bln&language=en_AE'
driver.get(mainurl)

while True:
    try:
        WebDriverWait(driver, 20).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'loading-mask'))
        )
        
        # Update the button locator to something more reliable (using XPath as an example)
        show_more_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ShowMoreButton")]'))
        )
        
        time.sleep(5)  # Use a smaller sleep and test the timing.
        show_more_button.click()
        
        # Wait for the product grid items to load
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a.Overlay__overlay__LloCU.ProductGridItem__overlay__IQ3Kw'))
        )
    except TimeoutException:
        print("End of content or timeout reached.")
        break
    except Exception as e:
        print(f"Error: {e}")
        break

# Parse the page content after clicking 'Show More'
soup = BeautifulSoup(driver.page_source, 'html.parser')
url_tags = soup.find_all('a', class_='Overlay__overlay__LloCU ProductGridItem__overlay__IQ3Kw')

for url_tag in url_tags:
    urls.append(f"https://www.amazon.sa{url_tag['href']}&language=en_AE")

driver.quit()
print(f'{len(urls)} URLs Retrieved')


End of content or timeout reached.
102 URLs Retrieved


In [37]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            if match:
                camera_info = match.group(1)
                break

        if color_row:
    # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        else:
    # If the color row doesn't exist, check for a 'selection' class
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = selection_row.get_text(strip=True)
                else:
                    color = None
            else:
                color = None



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': get_text_or_none(soup.find('span', class_='a-price-whole')),
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('SAR')
            Website.append('Amazon SA')

# Create a DataFrame to store the scraped data
df_apple = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_apple

Reviews section not found
Reviews section not found


,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Model,...,Wireless Provider,Cellular Technology,Camera,Color,Price,Price Before Promotion,Rate,URL,IMG,Reviews
0,Apple iPhone 15 (128 GB) - Black,Apple,iOS 17,None,None,128 GB,6.1 Inches,None,None,iPhone 15,...,Unlocked for All Carriers,5G,48MP,Black,"3,069.","SAR3,799.00",4.4 out of 5 stars,https://www.amazon.sa/Apple-iPhone-15-128-GB/d...,https://m.media-amazon.com/images/I/61bCKBrMVN...,"[{'reviewer_name': 'ممتاز', 'review_rating': '..."
1,Apple iPhone 15 (128 GB) - Pink,Apple,iOS 17,None,None,128 GB,6.1 Inches,None,None,iPhone 15,...,Unlocked for All Carriers,5G,48MP,Pink,"3,069.","SAR3,799.00",4.4 out of 5 stars,https://www.amazon.sa/Apple-iPhone-15-128-GB/d...,https://m.media-amazon.com/images/I/61bv+QKrIl...,"[{'reviewer_name': 'ممتاز', 'review_rating': '..."
2,Apple iPhone 15 (128 GB) - Blue,Apple,iOS 17,6 GB,None,128 GB,6.1 Inches,None,2 GHz,iPhone 15,...,Unlocked for All Carriers,5G,48MP,Blue,"3,009.","SAR3,799.00",4.4 out of 5 stars,https://www.amazon.sa/Apple-iPhone-15-128-GB/d...,https://m.media-amazon.com/images/I/61CjBAnIfG...,"[{'reviewer_name': 'ممتاز', 'review_rating': '..."
3,Apple iPhone 15 (128 GB) - Green,Apple,iOS 17,None,None,128 GB,6.1 Inches,None,None,iPhone 15,...,Unlocked for All Carriers,5G,48MP,Green,"3,198.","SAR3,799.00",4.4 out of 5 stars,https://www.amazon.sa/Apple-iPhone-15-128-GB/d...,https://m.media-amazon.com/images/I/61jDqNz2VC...,"[{'reviewer_name': 'ممتاز', 'review_rating': '..."
4,Apple iPhone 15 (256 GB) - Black,Apple,iOS 17,None,A13,256 GB,6.1 Inches,None,None,iPhone 15,...,Unlocked for All Carriers,5G,48MP,Black,"3,749.","SAR4,299.00",4.4 out of 5 stars,https://www.amazon.sa/Apple-iPhone-15-256-GB/d...,https://m.media-amazon.com/images/I/61bCKBrMVN...,"[{'reviewer_name': 'ممتاز', 'review_rating': '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,New Apple iPhone SE (64 GB) - Midnight (3rd ge...,Apple,iOS 15,None,None,64 GB,4.7 Inches,750 x 1334,None,iPhone,...,Unlocked for All Carriers,5G,None,Midnight,"1,949.","SAR2,049.00",4.3 out of 5 stars,https://www.amazon.sa/New-Apple-iPhone-SE-gene...,https://m.media-amazon.com/images/I/51f5qApDz7...,"[{'reviewer_name': 'Malek', 'review_rating': '..."
98,Apple iPhone 13 (256GB) - Blue,Apple,iOS 15,None,None,256 GB,6.1 Inches,1170 x 2532,None,iPhone,...,Unlocked for All Carriers,5G,None,Blue,"2,749.",None,4.5 out of 5 stars,https://www.amazon.sa/Apple-iPhone-13-256GB-Bl...,https://m.media-amazon.com/images/I/71xb2xkN5q...,"[{'reviewer_name': 'AHMED ELNOUBI', 'review_ra..."
99,Apple iPhone 12 (128GB) - White,Apple,iOS 14,4 GB,None,128 GB,6.1 Inches,3840 x 2160,2 GHz,iPhone 12,...,Cricket,5G,None,White,"1,799.","SAR3,049.00",4.4 out of 5 stars,https://www.amazon.sa/Apple-iPhone-12-128GB-Wh...,https://m.media-amazon.com/images/I/711wsjBtWe...,"[{'reviewer_name': 'Shereef', 'review_rating':..."
100,New Apple iPhone SE (128 GB) - Midnight (3rd Gen),Apple,iOS 15,None,None,128 GB,4.7 Inches,750 x 1334,None,iPhone,...,Unlocked for All Carriers,5G,None,Midnight,"2,319.",None,4.3 out of 5 stars,https://www.amazon.sa/New-Apple-iPhone-SE-128/...,https://m.media-amazon.com/images/I/61yiMwm0R8...,"[{'reviewer_name': 'Malek', 'review_rating': '..."


In [ ]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green","Emerald Green","Titanium Silver",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey","Glamorous Green","Flowing Silver",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_apple['Color'] = df_apple.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

Amazon SA Samsung

In [23]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

urls = []
driver = webdriver.Chrome()
mainurl = 'https://www.amazon.sa/stores/page/847D38A2-1744-4356-AF91-A1953BF6275D?ingress=2&visitId=b5249c96-cb9b-4310-aa3c-01e520e43b0d&ref_=ast_bln&language=en_AE'
driver.get(mainurl)

while True:
    try:
        WebDriverWait(driver, 20).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'loading-mask'))
        )
        
        # Update the button locator to something more reliable (using XPath as an example)
        show_more_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ShowMoreButton")]'))
        )
        
        time.sleep(5)  # Use a smaller sleep and test the timing.
        show_more_button.click()
        
        # Wait for the product grid items to load
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a.Overlay__overlay__LloCU.ProductGridItem__overlay__IQ3Kw'))
        )
    except TimeoutException:
        print("End of content or timeout reached.")
        break
    except Exception as e:
        print(f"Error: {e}")
        break

# Parse the page content after clicking 'Show More'
soup = BeautifulSoup(driver.page_source, 'html.parser')
url_tags = soup.find_all('a', class_='Overlay__overlay__LloCU ProductGridItem__overlay__IQ3Kw')

for url_tag in url_tags:
    urls.append(f"https://www.amazon.sa{url_tag['href']}&language=en_AE")

driver.quit()
print(f'{len(urls)} URLs Retrieved')


End of content or timeout reached.
220 URLs Retrieved


In [34]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            if match:
                camera_info = match.group(1)
                break

        if color_row:
    # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        else:
    # If the color row doesn't exist, check for a 'selection' class
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = selection_row.get_text(strip=True)
                else:
                    color = None
            else:
                color = None



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen') if soup.find('div', class_='a-section a-spacing-micro') else None),
            'Price Before Promotion' : get_text_or_none(soup.find('td', class_='a-span12 a-color-secondary a-size-base').find('span', class_='a-offscreen') if soup.find('td', class_='a-span12 a-color-secondary a-size-base') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('SAR')
            Website.append('Amazon SA')

# Create a DataFrame to store the scraped data
df_samsung = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_samsung

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found


,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Model,...,Wireless Provider,Cellular Technology,Camera,Color,Price,Price Before Promotion,Rate,URL,IMG,Reviews
0,"SAMSUNG Galaxy Z Fold6, AI Phone, 12GB RAM, 51...",SAMSUNG,Android,None,None,None,None,None,None,None,...,None,None,None,Navy,"SAR8,047.65","SAR8,198.00",5.0 out of 5 stars,https://www.amazon.sa/SAMSUNG-Storage-Android-...,https://m.media-amazon.com/images/I/714z7qd2lL...,"[{'reviewer_name': '🥰', 'review_rating': '5.0 ..."
1,"SAMSUNG Galaxy Z Fold6, AI Phone, 12GB RAM, 51...",SAMSUNG,Android,12 GB,None,512 GB,None,None,None,Galaxy Z Fold6,...,None,5G,None,Pink,"SAR8,057.00","SAR8,198.00",5.0 out of 5 stars,https://www.amazon.sa/SAMSUNG-Storage-Android-...,https://m.media-amazon.com/images/I/71kEZ9ut6h...,"[{'reviewer_name': '🥰', 'review_rating': '5.0 ..."
2,"SAMSUNG Galaxy Z Fold6, AI Phone, 12GB RAM, 51...",SAMSUNG,Android,None,None,None,None,None,None,None,...,None,None,None,Silver Shadow,"SAR8,013.29","SAR8,198.00",5.0 out of 5 stars,https://www.amazon.sa/SAMSUNG-Storage-Android-...,https://m.media-amazon.com/images/I/61IZfefAu8...,"[{'reviewer_name': '🥰', 'review_rating': '5.0 ..."
3,"SAMSUNG Galaxy Z Fold6, AI Phone, 12GB RAM, 25...",SAMSUNG,Android,None,None,None,None,None,None,None,...,None,None,None,Navy,"SAR6,846.51","SAR7,698.00",5.0 out of 5 stars,https://www.amazon.sa/SAMSUNG-Storage-Android-...,https://m.media-amazon.com/images/I/714z7qd2lL...,"[{'reviewer_name': '🥰', 'review_rating': '5.0 ..."
4,"SAMSUNG Galaxy Z Fold6, AI Phone, 12GB RAM, 25...",SAMSUNG,Android,None,None,None,None,None,None,None,...,None,None,None,Pink,"SAR7,556.98","SAR7,698.00",5.0 out of 5 stars,https://www.amazon.sa/SAMSUNG-Storage-Android-...,https://m.media-amazon.com/images/I/71kEZ9ut6h...,"[{'reviewer_name': '🥰', 'review_rating': '5.0 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,"Samsung Galaxy A05 LTE, Android Smartphone, Du...",SAMSUNG,Android 13.0,4 GB,A-Series,128 GB,6.7 Inches,720 x 1600,None,Galaxy A05,...,etisalat,None,None,Black,SAR398.00,None,4.0 out of 5 stars,https://www.amazon.sa/Samsung-Android-Smartpho...,https://m.media-amazon.com/images/I/71dYQdmhwF...,"[{'reviewer_name': 'Mohamed ibrahim', 'review_..."
216,"Samsung Galaxy A05, Android Smartphone, Dual S...",SAMSUNG,Android,4 GB,A-Series,128 GB,6.5 Inches,1080 x 2400,None,Galaxy A05,...,etisalat,None,None,Light Green,SAR371.00,None,4.0 out of 5 stars,https://www.amazon.sa/Samsung-Android-Smartpho...,https://m.media-amazon.com/images/I/71ve-OMxzy...,"[{'reviewer_name': 'Mohamed ibrahim', 'review_..."
217,"Samsung Galaxy A05, Android Smartphone, Dual S...",SAMSUNG,Android,4 GB,Snapdragon,64 GB,6.7 Inches,1080 x 2400,None,Galaxy A05,...,etisalat,None,None,Silver,SAR370.00,None,4.0 out of 5 stars,https://www.amazon.sa/Samsung-Android-Smartpho...,https://m.media-amazon.com/images/I/71-QzvIZnV...,"[{'reviewer_name': 'Mohamed ibrahim', 'review_..."
218,"Samsung Galaxy A05, Android Smartphone, Dual S...",SAMSUNG,Android,4 GB,Snapdragon,64 GB,6.7 Inches,720 x 1600,None,Galaxy A05,...,etisalat,None,None,Black,SAR350.00,None,4.0 out of 5 stars,https://www.amazon.sa/Samsung-Android-Smartpho...,https://m.media-amazon.com/images/I/71dYQdmhwF...,"[{'reviewer_name': 'Mohamed ibrahim', 'review_..."


In [ ]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green","Emerald Green","Titanium Silver",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey","Glamorous Green","Flowing Silver",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_samsung['Color'] = df_samsung.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

Amazon SA XIAOMI

In [42]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

urls = []
driver = webdriver.Chrome()
mainurl = 'https://www.amazon.sa/stores/page/935C5EB6-62FA-4BA0-B742-56910DB47392?ingress=2&visitId=82cee01f-8148-4e81-b3c1-8452e4561aa5&ref_=ast_bln&language=en_AE'
driver.get(mainurl)

while True:
    try:
        WebDriverWait(driver, 20).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'loading-mask'))
        )
        
        # Update the button locator to something more reliable (using XPath as an example)
        show_more_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ShowMoreButton")]'))
        )
        
        time.sleep(5)  # Use a smaller sleep and test the timing.
        show_more_button.click()
        
        # Wait for the product grid items to load
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a.Overlay__overlay__LloCU.ProductGridItem__overlay__IQ3Kw'))
        )
    except TimeoutException:
        print("End of content or timeout reached.")
        break
    except Exception as e:
        print(f"Error: {e}")
        break

# Parse the page content after clicking 'Show More'
soup = BeautifulSoup(driver.page_source, 'html.parser')
url_tags = soup.find_all('a', class_='Overlay__overlay__LloCU ProductGridItem__overlay__IQ3Kw')

for url_tag in url_tags:
    urls.append(f"https://www.amazon.sa{url_tag['href']}&language=en_AE")

driver.quit()
print(f'{len(urls)} URLs Retrieved')


End of content or timeout reached.
59 URLs Retrieved


In [62]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('td', class_='a-span12 a-color-secondary a-size-base').find('span', class_='a-offscreen') if soup.find('td', class_='a-span12 a-color-secondary a-size-base') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('SAR')
            Website.append('Amazon SA')

# Create a DataFrame to store the scraped data
df_xiaomi = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_xiaomi

Reviews section not found
Reviews section not found
Reviews section not found


,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Model,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,Xiaomi POCO X4 Pro 5G Dual SIM 8GB RAM 256GB 5...,Xiaomi,"MIUI 13, Android 11.0",8 GB,Snapdragon,256 GB,6.67 Inches,1080 x 2400,2.4 GHz,X4 Pro,...,None,None,"1,332.",None,SAR,3.9 out of 5 stars,https://www.amazon.sa/Xiaomi-POCO-256GB-Laser-...,https://m.media-amazon.com/images/I/71GG3PPhWK...,"[{'reviewer_name': 'user-MWAQASCH', 'review_ra...",Amazon SA
1,Xiaomi Poco X4 Pro 5G Dual Sim 6Gb Ram 128Gb 5...,Xiaomi,"MIUI 13, Android 11.0",6 GB,Snapdragon,128 GB,6.67 Inches,1080 x 2400,2.4 GHz,X4 Pro,...,None,None,"SAR1,297.70",None,SAR,4.1 out of 5 stars,https://www.amazon.sa/Xiaomi-Poco-128Gb-Laser-...,https://m.media-amazon.com/images/I/71GG3PPhWK...,"[{'reviewer_name': 'Ahmed', 'review_rating': '...",Amazon SA
2,Xiaomi Poco X4 Pro 5G Dual Sim 6Gb Ram 128Gb 5...,Xiaomi,"MIUI 13, Android 11.0",6 GB,Snapdragon,128 GB,6.67 Inches,2400 x 1080,2.4 GHz,X4 Pro,...,None,None,"1,258.",None,SAR,3.3 out of 5 stars,https://www.amazon.sa/Xiaomi-Poco-Dual-128Gb-Y...,https://m.media-amazon.com/images/I/71Q7BvFANL...,"[{'reviewer_name': 'Khaled saad', 'review_rati...",Amazon SA
3,"Redmi 12C (Graphite Gray 6GB RAM, 128 Storage)...",Xiaomi,Android 12.0,6 GB,MediaTek Helio,128 GB,6.71 Inches,720 x 1280,2.84,Redmi 12C,...,None,None,SAR456.78,None,SAR,4.4 out of 5 stars,https://www.amazon.sa/Redmi-12C-Graphite-Gray-...,https://m.media-amazon.com/images/I/51UtIuo-sU...,"[{'reviewer_name': 'Sirajul Islam', 'review_ra...",Amazon SA
4,Redmi 12 5G Dual SIM Midnight Black 8GB RAM 25...,Xiaomi,Android 13.0,8 GB,Snapdragon,256 GB,6.79 Inches,1080 x 2460,"2.0, 2.2 GHz",Redmi 12 5G,...,None,None,"2,219.",None,SAR,3.5 out of 5 stars,https://www.amazon.sa/Redmi-Dual-Midnight-Blac...,https://m.media-amazon.com/images/I/61b67F9qoO...,"[{'reviewer_name': 'Indra Monaram', 'review_ra...",Amazon SA
5,Xiaomi Redmi 12 5G Dual SIM Polar Silver 8GB R...,Xiaomi,Android 13.0,8 GB,Snapdragon,256 GB,6.79 Inches,1080 x 2460,"2.0, 2.2 GHz",Redmi 12 5G,...,None,None,599.,None,SAR,3.9 out of 5 stars,https://www.amazon.sa/Xiaomi-Redmi-Polar-Silve...,https://m.media-amazon.com/images/I/61vBh7yhZr...,"[{'reviewer_name': 'Zahiduzzaman Siddiquee', '...",Amazon SA
6,Redmi 12 5G Dual SIM mid-night black 8GB RAM 2...,Xiaomi,Android 13.0,8 GB,Snapdragon,256 GB,6.79 Inches,1080 x 2460,"2.0, 2.2 GHz",Redmi 12 5G,...,None,None,"2,219.",None,SAR,4.1 out of 5 stars,https://www.amazon.sa/Redmi-Dual-mid-night-bla...,https://m.media-amazon.com/images/I/71zlzrTdzS...,"[{'reviewer_name': 'Kiezle Jacinto', 'review_r...",Amazon SA
7,Xiaomi Redmi Note 12 Dual Sim 8GB RAM 256GB 4G...,Xiaomi,Android 13.0,8 GB,Snapdragon,256 GB,6.67 Inches,2400 x 1080,2.8,Redmi Note 12,...,None,None,699.,None,SAR,4.4 out of 5 stars,https://www.amazon.sa/Xiaomi-Redmi-Note-Dual-2...,https://m.media-amazon.com/images/I/71YsGlqN4X...,"[{'reviewer_name': 'seraj', 'review_rating': '...",Amazon SA
8,Redmi Note 12 Dual SIM Onyx Gray 8GB RAM 256GB...,Xiaomi,Android 13.0,8 GB,Snapdragon,256 GB,6.67 Inches,2400 x 1080,2.8,Redmi Note 12,...,None,None,699.,None,SAR,4.4 out of 5 stars,https://www.amazon.sa/Redmi-Note-Dual-Onyx-256...,https://m.media-amazon.com/images/I/71iv8KcK4w...,"[{'reviewer_name': 'seraj', 'review_rating': '...",Amazon SA
9,"Xiaomi Redmi Note 12, 6GB RAM, 128 GB, Onyx Gr...",Xiaomi,Android 13.0,6 GB,Snapdragon,128 GB,6.67 Inches,1080 x 2400,2.4 GHz,Redmi Note 12 6+128 GB Onyx Gray,...,None,None,579.,None,SAR,4.1 out of 5 stars,https://www.amazon.sa/Xiaomi-Redmi-Note-Onyx-B...,https://m.media-amazon.com/images/I/61p79A67kP...,"[{'reviewer_name': 'Abdallah Al-Otaibi', 'revi...",Amazon SA


In [63]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green","Emerald Green","Titanium Silver",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey","Glamorous Green","Flowing Silver",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_xiaomi['Color'] = df_xiaomi.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

Amazon SA HONOR

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Firefox()

urls = []
for page in range(1, 6):
    url = f'https://www.amazon.sa/s?i=electronics&bbn=16966419031&rh=n%3A16966419031%2Cp_n_feature_six_browse-bin%3A16967122031%2Cp_n_feature_eleven_browse-bin%3A27361539031%2Cp_89%3AHONOR&dc&page={page}&language=en_AE'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    time.sleep(2)
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.sa{url_tag['href']}&language=en_AE")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

119 Product URLs retrieved.


In [7]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('td', class_='a-span12 a-color-secondary a-size-base').find('span', class_='a-offscreen') if soup.find('td', class_='a-span12 a-color-secondary a-size-base') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('SAR')
            Website.append('Amazon SA')

# Create a DataFrame to store the scraped data
df_honor = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_honor

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews sect

,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Model,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,Honor Magic Vs Dual-SIM 512GB ROM + 12GB RAM (...,HONOR,Android 13.0,12 GB,Snapdragon,512 GB,7.9 Inches,720 x 1280,"2.5, 1.8 GHz",Magic Vs,...,None,None,147.,None,SAR,Previous page of related Sponsored Products,https://www.amazon.sa/-/en/Honor-Dual-SIM-Fact...,https://m.media-amazon.com/images/I/61vTxWTmlY...,[],Amazon SA
1,"HUAWEI 9 LITE Honor 9 Lite Dual SIM - 32GB, 3G...",None,None,None,None,None,None,None,None,None,...,None,None,"SAR1,709.00",None,SAR,4.5 out of 5 stars,https://www.amazon.sa/-/en/HUAWEI-LITE-Honor-L...,https://m.media-amazon.com/images/I/81tTAs93V7...,[],Amazon SA
2,Honor Magic 4 Lite 4G Smartphone 6 + 128 GB An...,HONOR,Android 11.0,8 GB,Snapdragon,128 GB,6.81 Inches,1080 x 2388,"2.2, 1.7 GHz",Magic4,...,None,None,"SAR1,900.00",None,SAR,4.4 out of 5 stars,https://www.amazon.sa/-/en/Smartphone-Android-...,https://m.media-amazon.com/images/I/511fBkfpLB...,[],Amazon SA
3,"Honor X8 17 cm (6.7"") Double SIM Android 11 4G...",HONOR,Android 8.1,6 GB,Snapdragon,6 GB,6.7 Inches,None,2 GHz,X8,...,None,None,SAR889.54,None,SAR,3.8 out of 5 stars,https://www.amazon.sa/-/en/Honor-Double-Androi...,https://m.media-amazon.com/images/I/51GzWdEhjq...,[],Amazon SA
4,Honor 90 Dual-SIM 512GB ROM + 12GB RAM (Only G...,HONOR,None,None,Snapdragon,512 GB,6.5 Inches,None,"2.36, 2.5, 1.8 GHz",None,...,None,None,"SAR3,165.00",None,SAR,5.0 out of 5 stars,https://www.amazon.sa/-/en/Dual-SIM-Factory-Un...,https://m.media-amazon.com/images/I/71o22oBFlN...,[],Amazon SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Honor 8X Single-SIM 64GB (GSM Only | No CDMA) ...,HONOR,Android 8.1,4 GB,None,64 GB,6.5 Inches,None,"2.2, 1.7 GHz",8X,...,None,None,"SAR2,425.00",None,SAR,4.5 out of 5 stars,https://www.amazon.sa/-/en/Honor-Single-SIM-Fa...,https://m.media-amazon.com/images/I/91IvQtcikm...,[],Amazon SA
115,Honor X7 Dual SIM Ocean Blue 6GB RAM 128GB 4G LTE,HONOR,"Android R + MagicUI 4.2, Android 11.0",6 GB,Snapdragon,128 GB,6.74 Inches,720 x 1600,"1.9, 2.4 GHz",X7,...,None,None,678.,None,SAR,3.6 out of 5 stars,https://www.amazon.sa/-/en/Honor-Dual-Ocean-Bl...,https://m.media-amazon.com/images/I/612GlpnCOn...,"[{'reviewer_name': 'abdoullah alamoudi', 'revi...",Amazon SA
116,HONOR 200 Pro 5G Dual SIM 12GB RAM + 512 GB Bl...,HONOR,"Android 14, Android",None,None,512 GB,6.78 Inches,None,None,HONOR 200 Pro,...,50MP,Black,"2,488.",None,SAR,Previous page,https://www.amazon.sa/-/en/HONOR-Dual-Black-Mi...,https://m.media-amazon.com/images/I/41oKx1kRVD...,[],Amazon SA
117,Honor 10 Lite Dual Sim 64Gb Factory Unlocked 4...,HONOR,Android 9.0,3 GB,None,64 GB,6.21 Inches,1080 x 2340,None,Honor 10,...,None,None,"SAR1,999.00",None,SAR,4.4 out of 5 stars,https://www.amazon.sa/-/en/Factory-Unlocked-Sm...,https://m.media-amazon.com/images/I/61NwyJg1xy...,[],Amazon SA


In [8]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green","Emerald Green","Titanium Silver",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey","Glamorous Green","Flowing Silver",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_honor['Color'] = df_honor.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

Amazon SA NOKIA

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Firefox()

urls = []
for page in range(1, 6):
    url = f'https://www.amazon.sa/s?i=electronics&bbn=16966419031&rh=n%3A16966419031%2Cp_89%3ANokia%2Cp_n_feature_six_browse-bin%3A16967122031%2Cp_n_feature_eleven_browse-bin%3A27361539031&dc&page={page}&language=en_AE'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    time.sleep(2)
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.sa{url_tag['href']}&language=en_AE")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

109 Product URLs retrieved.


In [3]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('td', class_='a-span12 a-color-secondary a-size-base').find('span', class_='a-offscreen') if soup.find('td', class_='a-span12 a-color-secondary a-size-base') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        # print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('SAR')
            Website.append('Amazon SA')

# Create a DataFrame to store the scraped data
df_nokia = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_nokia

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews sect

,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Model,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,"NOKIA C2 2E, 32GB+2GB, DS, GREY",Nokia,Android 11.0,32 GB,A-Series,32 GB,5.7 Inches,480 x 960,1.5 GHz,NOKIA C2 2E,...,None,None,299.,None,SAR,3.6 out of 5 stars,https://www.amazon.sa/-/en/NOKIA-C2-32GB-2GB-G...,https://m.media-amazon.com/images/I/810yqLjxjx...,"[{'reviewer_name': 'سلطان', 'review_rating': '...",Amazon SA
1,Nokia 1.4 D.Sim 1/16GB - Charcoal,Nokia,Android 10.0,2 GB,Snapdragon,16 GB,6.51 Inches,None,2 GHz,1.4,...,None,None,"SAR1,355.00",None,SAR,5.0 out of 5 stars,https://www.amazon.sa/-/en/Nokia-1-4-D-Sim-16G...,https://m.media-amazon.com/images/I/7198+l23II...,[],Amazon SA
2,Nokia C12 Dual SIM 64GB ROM + 2GB RAM Factory ...,Nokia,Android,2 GB,Snapdragon,64 GB,6.3 Inches,720 x 1600,"1.2, 2.35, 1.6 GHz",C12,...,None,None,"SAR1,105.00",None,SAR,Previous page of related Sponsored Products,https://www.amazon.sa/-/en/Nokia-Factory-Unloc...,https://m.media-amazon.com/images/I/71WAn3PSaz...,[],Amazon SA
3,Nokia 5.4 4G Dual Sim Smartphone with a 6.39” ...,Nokia,Android 11.0,4 GB,Snapdragon,128 GB,6.39 Inches,720 x 1560,2 GHz,Nokia 5.4 4G,...,None,None,535.,None,SAR,3.7 out of 5 stars,https://www.amazon.sa/-/en/Nokia-Smartphone-Qu...,https://m.media-amazon.com/images/I/81IpIjYRVm...,"[{'reviewer_name': 'Abdulsalam', 'review_ratin...",Amazon SA
4,"Nokia G11 Plus, 4GB/64GB, Grey",Nokia,Android 12.0,4 GB,Snapdragon,4 GB,6.52 Inches,720 x 1600,None,G11 Plus,...,None,nano,SAR441.02,None,SAR,4.2 out of 5 stars,https://www.amazon.sa/-/en/Nokia-G11-Plus-64GB...,https://m.media-amazon.com/images/I/81fvRSokWV...,"[{'reviewer_name': 'Sahar', 'review_rating': '...",Amazon SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Nokia C21 Plus Dual-Sim 32GB ROM + 2GB RAM (GS...,Nokia,None,2 GB,Snapdragon,2 GB,6.5 Inches,720 x 1600,"1.2, 1.6 GHz",C21 Plus,...,None,None,SAR965.00,None,SAR,4.1 out of 5 stars,https://www.amazon.sa/-/en/Nokia-Dual-Sim-Fact...,https://m.media-amazon.com/images/I/8161jfuxrv...,[],Amazon SA
105,"Nokia 5.3 Android Smartphone, 4GB RAM, 64GB Me...",Nokia,Android 10.0,4 GB,Snapdragon,64 GB,6.55 Inches,720 x 1600,None,Nokia 5.3 464,...,None,None,"SAR1,885.00",None,SAR,3.6 out of 5 stars,https://www.amazon.sa/-/en/Nokia-Android-Smart...,https://m.media-amazon.com/images/I/61KfjV+40Q...,"[{'reviewer_name': 'Md Mukhlesur Rahman', 'rev...",Amazon SA
106,Nokia C2 2nd Edition Dual-SIM 32GB ROM + 2GB R...,Nokia,Android,2 GB,Snapdragon,32 GB,5.7 Inches,480 x 960,1.5 GHz,C2 2nd Edition,...,None,None,"SAR1,155.00",None,SAR,5.0 out of 5 stars,https://www.amazon.sa/-/en/Nokia-Dual-SIM-Fact...,https://m.media-amazon.com/images/I/615XyiBJm3...,[],Amazon SA
107,Nokia Oppo Reno7 5G Dual-SIM 256GB ROM + 8GB R...,Nokia,Android 11.0,8 GB,Snapdragon,256 GB,6.43 Inches,None,"2.0, 2.4 GHz",Reno7 5G,...,None,None,"SAR3,235.00",None,SAR,5.0 out of 5 stars,https://www.amazon.sa/-/en/Nokia-Dual-SIM-Unlo...,https://m.media-amazon.com/images/I/61drifEnVn...,[],Amazon SA


In [4]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green","Emerald Green","Titanium Silver",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey","Glamorous Green","Flowing Silver",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_nokia['Color'] = df_nokia.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

Amazon SA HUAWEI

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Firefox()

urls = []
for page in range(1, 8):
    url = f'https://www.amazon.sa/s?i=electronics&bbn=16966419031&rh=n%3A16966419031%2Cp_n_feature_six_browse-bin%3A16967122031%2Cp_n_feature_eleven_browse-bin%3A27361539031%2Cp_89%3AHUAWEI&dc&page={page}&language=en_AE'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    time.sleep(2)
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.sa{url_tag['href']}&language=en_AE")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

165 Product URLs retrieved.


In [11]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            if match:
                camera_info = match.group(1)
                break

        if color_row:
    # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        else:
    # If the color row doesn't exist, check for a 'selection' class
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = selection_row.get_text(strip=True)
                else:
                    color = None
            else:
                color = None



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen') if soup.find('div', class_='a-section a-spacing-micro') else None),
            'Price Before Promotion' : get_text_or_none(soup.find('td', class_='a-span12 a-color-secondary a-size-base').find('span', class_='a-offscreen') if soup.find('td', class_='a-span12 a-color-secondary a-size-base') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('SAR')
            Website.append('Amazon SA')

# Create a DataFrame to store the scraped data
df_huawei = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_huawei

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews sect

,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Model,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,"HUAWEI nova 12i Smart Phone, Dual SIM Mobile P...",HUAWEI,EMUI 14,8 GB,Snapdragon,256 GB,6.7 Inches,None,None,HUAWEI nova 12i,...,None,Green,None,None,SAR,3.4 out of 5 stars,https://www.amazon.sa/-/en/HUAWEI-nova-12i-6-7...,https://m.media-amazon.com/images/I/81RuDGfNos...,"[{'reviewer_name': 'mahmoud', 'review_rating':...",Amazon SA
1,"Huawei Y6S Dual SIM - 6.9 Inch, 64 GB, 3 GB RA...",HUAWEI,Android 9.0,3 GB,MediaTek Helio,64 GB,6.09 Inches,720 x 1560,None,Y6S,...,None,None,"SAR1,769.00",None,SAR,4.3 out of 5 stars,https://www.amazon.sa/-/en/Huawei-Y6S-Dual-SIM...,https://m.media-amazon.com/images/I/61hmN0LxFC...,"[{'reviewer_name': 'محمد المنقري', 'review_rat...",Amazon SA
2,"HUAWEI nova 12 SE Smart Phone, Dual SIM, 6.67""...",HUAWEI,EMUI 14,8 GB,Snapdragon,256 GB,6.7 Inches,None,None,HUAWEI nova 12 SE,...,None,Green,None,None,SAR,3.6 out of 5 stars,https://www.amazon.sa/-/en/HUAWEI-nova-12-SE-S...,https://m.media-amazon.com/images/I/7141MYuhbu...,"[{'reviewer_name': 'Louie', 'review_rating': '...",Amazon SA
3,"HUAWEI Y9 2018 (FLA-LX3) 5.9"" Dual Camera 13MP...",HUAWEI,Android,3 GB,Snapdragon,32 GB,5.9 Inches,None,1.8 GHz,HUAWEI Y9,...,None,None,SAR884.00,None,SAR,3.8 out of 5 stars,https://www.amazon.sa/-/en/HUAWEI-2018-FLA-LX3...,https://m.media-amazon.com/images/I/61zDSBYl7Z...,"[{'reviewer_name': 'Mohammad Naushad Alam', 'r...",Amazon SA
4,Huawei nova 9 SE Dual-SIM 128GB ROM + 8GB RAM ...,HUAWEI,Android 11.0,8 GB,Snapdragon,128 GB,6.78 Inches,1080 x 2388,"1.9, 2.4 GHz",nova 9 SE,...,None,None,"SAR2,885.00",None,SAR,3.6 out of 5 stars,https://www.amazon.sa/-/en/Huawei-Dual-SIM-Fac...,https://m.media-amazon.com/images/I/51uuB3y3wP...,[],Amazon SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,HUAWEI Mate Xs 2 + (Freebuds Pro 2 White) - 7....,HUAWEI,EMUI 12,8 GB,Snapdragon,512 GB,7.8 Inches,3840 x 2160,None,Mate Xs 2 + Freebuds Pro 2,...,None,None,"SAR7,650.79",None,SAR,Previous page of related Sponsored Products,https://www.amazon.sa/-/en/HUAWEI-Mate-Freebud...,https://m.media-amazon.com/images/I/61enp5gulr...,[],Amazon SA
161,"Huawei P20 Pro 128GB Dual-SIM (GSM Only, No CD...",HUAWEI,Android 8.1,6 GB,Snapdragon,128 GB,6.1 Inches,1080 x 2240,"1.8, 2.4 GHz",HUAWEI P20 Pro,...,None,Black,"SAR3,195.00",None,SAR,4.1 out of 5 stars,https://www.amazon.sa/-/en/Huawei-Dual-SIM-Fac...,https://m.media-amazon.com/images/I/61FRUed8ml...,[],Amazon SA
162,"HUAWEI Pura 70 Ultra Smartphone, Dual SIM, 16 ...",HUAWEI,EMUI 14.2,16 GB,Snapdragon,512 GB,6.8 Inches,None,3.48 THz,HUAWEI Pura 70 Ultra,...,None,None,None,None,SAR,Previous page,https://www.amazon.sa/-/en/HUAWEI-Smartphone-L...,https://m.media-amazon.com/images/I/71F0uxV4bl...,[],Amazon SA
163,HUAWEI P30 128GB 6GB RAM ELE-L29 LTE Factory U...,HUAWEI,Android 9.0,6 GB,None,128 GB,6.3 Inches,1080 x 2340,"2.6, 1.92, 1.8 GHz",HUAWEI P30,...,None,None,"SAR5,492.00",None,SAR,4.6 out of 5 stars,https://www.amazon.sa/-/en/HUAWEI-ELE-L29-Fact...,https://m.media-amazon.com/images/I/61TsiAtMVQ...,[],Amazon SA


In [12]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green","Emerald Green","Titanium Silver",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey","Glamorous Green","Flowing Silver",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_huawei['Color'] = df_huawei.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

Final Result (Before Cleaning)

In [ ]:
df = pd.concat([df_nokia, df_samsung, df_apple, df_xiaomi, df_honor, df_huawei], ignore_index=True)

In [ ]:
df.to_csv('Final Amazon SA.csv', index = False)